In [55]:
import pandas as pd
import numpy as np

In [86]:
import copy

In [56]:
train = pd.read_csv('./train.csv')

In [57]:
test = pd.read_csv('./test.csv')

In [58]:
train

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,Resort Hotel,0,342,2015,July,27,1,0,0,...,No Deposit,NaN,NaN,0,Transient,-6.305161,0,0,Check-Out,2015-07-01
1,1,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6.0,NaN,0,Transient,75.052227,0,0,Check-Out,2015-07-03
2,2,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6.0,NaN,0,Transient,74.546401,0,0,Check-Out,2015-07-03
3,3,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6.0,NaN,0,Transient,76.376288,0,0,Check-Out,2015-07-03
4,4,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6.0,NaN,0,Transient,49.411647,0,0,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,91526,Resort Hotel,1,19,2017,March,13,31,0,2,...,No Deposit,250.0,NaN,0,Transient,79.223571,0,1,Canceled,2017-03-22
91527,91527,Resort Hotel,0,28,2017,March,13,31,0,2,...,No Deposit,NaN,NaN,0,Transient,-6.822102,0,0,Check-Out,2017-04-02
91528,91528,Resort Hotel,0,2,2017,March,13,31,0,1,...,No Deposit,NaN,NaN,0,Transient,90.814554,0,2,Check-Out,2017-04-01
91529,91529,Resort Hotel,0,30,2017,March,13,31,3,7,...,No Deposit,250.0,NaN,0,Transient,38.135565,0,1,Check-Out,2017-04-10


In [59]:
train.drop(['country'],axis=1,inplace=True)

In [60]:
train.drop(['company'],axis=1,inplace=True)

In [61]:
train.drop(['reservation_status','reservation_status_date'],axis=1,inplace=True)

In [62]:
train['children'].fillna(0,inplace=True)
train['agent'].fillna(0,inplace=True)
train.hotel =train.hotel.astype('category')
train.children = train["children"].astype(int)
train.agent = train['agent'].astype(int)
train['meal'].replace('Undefined','SC',inplace=True)

In [63]:
test.drop(['country'],axis=1,inplace=True)

In [64]:
test.drop(['company'],axis=1,inplace=True)

In [65]:
test['children'].fillna(0,inplace=True)
test['agent'].fillna(0,inplace=True)
test.hotel =test.hotel.astype('category')
test.children = test["children"].astype(int)
test.agent = test['agent'].astype(int)
test['meal'].replace('Undefined','SC',inplace=True)

In [66]:
set(test.columns) ^ set(train.columns)

{'adr', 'is_canceled'}

In [67]:
x_tmp = train.drop(['adr','is_canceled'],axis=1)

In [68]:
y_tmp = train['is_canceled']

In [73]:
set(x_tmp.columns) ^ set(test.columns)

set()

In [70]:
from sklearn.ensemble import RandomForestClassifier
x_tmp = pd.get_dummies(data=x_tmp,columns=["hotel","arrival_date_month","meal","market_segment",
"distribution_channel","reserved_room_type","deposit_type","customer_type","assigned_room_type"])

In [71]:
test = pd.get_dummies(data=test,columns=["hotel","arrival_date_month","meal","market_segment",
"distribution_channel","reserved_room_type","deposit_type","customer_type","assigned_room_type"])

In [72]:
test['arrival_date_month_February'] = 0 
test['market_segment_Undefined'] = 0
test['arrival_date_month_November'] = 0
test['distribution_channel_Undefined'] = 0
test['arrival_date_month_January'] = 0
test['arrival_date_month_December'] = 0
test['reserved_room_type_L'] = 0
test['arrival_date_month_October'] = 0
test['arrival_date_month_September'] = 0
test['arrival_date_month_March'] = 0
test['assigned_room_type_L'] = 0

In [74]:
test=test[x_tmp.columns]

In [75]:
X = x_tmp.values

In [76]:
X.shape

(91531, 78)

In [77]:
Y = y_tmp.values

In [78]:
Y.shape

(91531,)

In [79]:
clf = RandomForestClassifier(oob_score=True)

In [80]:
clf.fit(X,Y)

RandomForestClassifier(oob_score=True)

In [81]:
clf.oob_score_

0.8924735881832385

In [82]:
test_cancel_predict = clf.predict(test)

In [83]:
test_cancel_predict

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [135]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [136]:
test['is_canceled'] = test_cancel_predict

In [137]:
def combine_arrival_date(data):
    #將arrival_date做出來
    x_data = copy.deepcopy(data)
    day_to_str = {}
    for i in range(1,32):
        if i<10:
            day_to_str[str(i)]='0'+str(i)
        else:
            day_to_str[str(i)]=str(i)
    x_data['arrival_date_year'] = x_data['arrival_date_year'].astype(str)
    x_data['arrival_date_month'] = x_data['arrival_date_month'].map({'January':'01','February':'02','March':'03','April':'04','May':'05','June':'06','July':'07','August':'08','September':'09','October':'10','November':'11','December':'12'})
    x_data['arrival_date_day_of_month'] = x_data['arrival_date_day_of_month'].astype(str).map(day_to_str)
    x_data['arrival_date'] = x_data['arrival_date_year'] + '-' + x_data['arrival_date_month'] + '-' + x_data['arrival_date_day_of_month']
    # x_data.drop(['arrival_date_year','arrival_date_day_of_month'],axis=1,inplace=True)
    return x_data

In [138]:
def preprocess_x(x_data):
    # 丟掉 canceled 且 no deposit 的資料
    x_data.drop(x_data[(x_data['is_canceled']==1)&(x_data['deposit_type']=='No Deposit')].index,inplace=True)
    x_data = combine_arrival_date(x_data)
    #x_data['stays_total_nights'] = x_data['stays_in_week_nights'] + x_data['stays_in_weekend_nights']
    #x_data['booking_total_revenue'] = (x_data['stays_total_nights']+1)*x_data['adr']
#     train_booking_total_tmp = x_data.groupby('arrival_date').sum()
#     train_booking_total_tmp['arrival_date'] = train_booking_total_tmp.index
#     train_booking_total = train_booking_total_tmp[['arrival_date','booking_total_revenue']]
#     x = train_booking_total.set_index('arrival_date').values
#     y = y_data.set_index('arrival_date').values
#     y = np.reshape(y,(640,))
    return x_data

In [139]:
# function
def pre_adr_model(train,test):
    booking_total = preprocess_x(train)
    test = preprocess_x(test)
    # 丟掉一些沒用到的 features
    booking_total.drop(['country','company','is_canceled','reservation_status','reservation_status_date','adr'],axis=1,inplace=True)
    test.drop(['country','company','is_canceled'],axis=1,inplace=True)
    # children 補眾數(補 0)
    booking_total['children'].fillna(0,inplace=True)
    # agent 沒有的補 0
    # booking_total['agent'].fillna(0,inplace=True)
    # test['agent'].fillna(0,inplace=True)
    booking_total.drop(['agent'],axis=1,inplace=True)
    test.drop(['agent'],axis=1,inplace=True)

    # datatype 轉成一樣
    booking_total.hotel =booking_total.hotel.astype('category')
    test.hotel = test.hotel.astype('category')
    booking_total.children = booking_total["children"].astype(int)
    test.children = test["children"].astype(int)
    # meal 的 undefined 和 SC 是一樣的
    booking_total['meal'].replace('Undefined','SC',inplace=True)
    test['meal'].replace('Undefined','SC',inplace=True)
    X_booking = pd.get_dummies(data=booking_total,columns=["hotel","arrival_date_week_number","arrival_date_day_of_month","arrival_date_month","meal","market_segment",
    "distribution_channel","reserved_room_type","assigned_room_type","deposit_type","customer_type"])
    test = pd.get_dummies(data=test,columns=["hotel","arrival_date_week_number","arrival_date_day_of_month","arrival_date_month","meal","market_segment",
    "distribution_channel","reserved_room_type","assigned_room_type","deposit_type","customer_type"])
    X_booking['assigned_room_type_P'] = 0
    X_booking['reserved_room_type_P'] = 0
    test['reserved_room_type_L'] = 0
    test['arrival_date_month_01'] = 0
    test['arrival_date_month_02'] = 0
    test['arrival_date_month_03'] = 0
    test['arrival_date_month_09'] = 0
    test['arrival_date_month_10'] = 0
    test['arrival_date_month_11'] = 0
    test['arrival_date_month_12'] = 0
    test['distribution_channel_Undefined'] = 0
    test['arrival_date_week_number_1'] = 0
    test['arrival_date_week_number_10'] = 0
    test['arrival_date_week_number_11'] = 0
    test['arrival_date_week_number_12'] = 0
    test['arrival_date_week_number_2'] = 0
    test['arrival_date_week_number_3'] = 0
    test['arrival_date_week_number_36'] = 0
    test['arrival_date_week_number_37'] = 0
    test['arrival_date_week_number_38'] = 0
    test['arrival_date_week_number_39'] = 0
    test['arrival_date_week_number_4'] = 0
    test['arrival_date_week_number_40'] = 0
    test['arrival_date_week_number_41'] = 0
    test['arrival_date_week_number_42'] = 0
    test['arrival_date_week_number_43'] = 0
    test['arrival_date_week_number_44'] = 0
    test['arrival_date_week_number_45'] = 0
    test['arrival_date_week_number_46'] = 0
    test['arrival_date_week_number_47'] = 0
    test['arrival_date_week_number_48'] = 0
    test['arrival_date_week_number_49'] = 0
    test['arrival_date_week_number_5'] = 0
    test['arrival_date_week_number_50'] = 0
    test['arrival_date_week_number_51'] = 0
    test['arrival_date_week_number_52'] = 0
    test['arrival_date_week_number_53'] = 0
    test['arrival_date_week_number_6'] = 0
    test['arrival_date_week_number_7'] = 0
    test['arrival_date_week_number_8'] = 0
    test['arrival_date_week_number_9'] = 0
    X_booking.drop(['ID','arrival_date_year'],axis=1,inplace=True)
    test.drop(['ID','arrival_date_year'],axis=1,inplace=True)
    return X_booking,test

In [140]:
train,test=pre_adr_model(train,test)

In [141]:
set(train.columns) ^ set(test.columns)

set()

In [142]:
train_new = train.groupby('arrival_date').sum()
test_new = test.groupby('arrival_date').sum()

In [143]:
train_count = train.groupby('arrival_date').count()
test_count = test.groupby('arrival_date').count()

In [144]:
train_new['total'] = train_count['lead_time']
test_new['total'] = test_count['lead_time']

In [149]:
for i in range(1,54):
    item = 'arrival_date_week_number_%s'%i
    train_new[item] = train_new[item].astype(bool)
    train_new[item] = train_new[item].astype(int)
    test_new[item] = test_new[item].astype(bool)
    test_new[item] = test_new[item].astype(int)
for i in range(1,31):
    if i < 10:
        item = 'arrival_date_day_of_month_0%s'%i
    else:
        item = 'arrival_date_day_of_month_%s'%i
    train_new[item] = train_new[item].astype(bool)
    train_new[item] = train_new[item].astype(int)
    test_new[item] = test_new[item].astype(bool)
    test_new[item] = test_new[item].astype(int)
for i in range(1,12):
    if i < 10:
        item = 'arrival_date_month_0%s'%i
    else:
        item = 'arrival_date_month_%s'%i
    train_new[item] = train_new[item].astype(bool)
    train_new[item] = train_new[item].astype(int)
    test_new[item] = test_new[item].astype(bool)
    test_new[item] = test_new[item].astype(int)

In [151]:
train_new.to_csv('./x_tmp.csv')

In [152]:
train.shape

(70457, 156)

In [153]:
train_new.to_csv('./train_new.csv')
test_new.to_csv('./test_new.csv')